In [17]:
import numpyro
import numpyro.distributions as dist
import jax.random as random
import einops
import jax
import jax.numpy as jnp


def get_example_image(image_shape, sv_r, indexes_of_high, mean_0,var_0,mean_1,var_1,rng_key):
    """
    we will create a simulation of the image with supervoxel segmentation each super voxel 
    at first extremely simplified case with square supervoxels each filled with predefined gausiians output
    we will generate couple of such and try to make a graph model that would recognize the diffrences
    indexes_of_high - will be a set of tuples indicating w and  h coordinates
    """ 
    #1)create empy image
    image= jnp.zeros(image_shape)
    #2) reshape to get as las two dims sv size
    image= einops.rearrange(image, '(w sw) (h sh) ->w h sw sh', sw=sv_r,sh=sv_r )
    image=numpyro.sample('mu', dist.Normal(mean_0,var_0), sample_shape=image.shape,rng_key=rng_key)
    #populate the sv areas with goussian noise of mean and var 1 if index is in indexes_of_high and mean and var 0 otherwise
    rng_key_new, old= random.split(rng_key)
    for coord in indexes_of_high:
        rng_key_new, old= random.split(rng_key_new)
        x,y=coord
        image.at[x,y,:,:].set(numpyro.sample('mu_1', dist.Normal(mean_1,var_1), sample_shape=(sv_r,sv_r),rng_key=rng_key_new))

    #reshape back to original image shape
    image= einops.rearrange(image, 'w h sw sh -> (w sw) (h sh) ', sw=sv_r,sh=sv_r )
    return image

rng_key = random.PRNGKey(3)
rng_key_new,old = random.split(rng_key)

mean_0 =1
var_0=1
mean_1=2
var_1=0.5
image_shape=(64,64)
sv_r=8
indexes_of_high=[(0,0), (1,1), (2,2)]
get_example_image(image_shape, sv_r, indexes_of_high, mean_0,var_0,mean_1,var_1,old)
v_get_example_image=jax.vmap(get_example_image,in_axes=(None,None,None,None,None,None,None,0))

#                             ,in_axes=(0, 0,0)

In [19]:
def build_example_dataset():
    """ 
    using get_example_image  we will create multiple instances of diffrent images - we will save images
    task here will be graph level predictions 
    """
    num_per_class=50
    mean_0 =1
    var_0=1
    mean_1=2
    var_1=0.5
    image_shape=(64,64)
    sv_r=8
    a=v_get_example_image(image_shape, sv_r, [(0,0), (1,1), (2,2), (3,3)], mean_0,var_0,mean_1,var_1,random.split(rng_key_new,num_per_class  ))
    b=v_get_example_image(image_shape, sv_r, [(0,0), (0,1), (0,2), (0,3)], mean_0,var_0,mean_1,var_1,random.split(rng_key_new,num_per_class  ))
    c=v_get_example_image(image_shape, sv_r, [(0,0), (1,1), (0,1), (1,1)], mean_0,var_0,mean_1,var_1,random.split(rng_key_new,num_per_class  ))
    images=jnp.concatenate([a,b,c],axis=0)
    labels_a= jnp.ones(num_per_class)
    labels_b= jnp.ones(num_per_class)*2
    labels_c= jnp.ones(num_per_class)*3
    labels= jnp.concatenate([labels_a,labels_b, labels_c])
    return images,labels

dummy_datase=build_example_dataset()
dummy_datase[0].shape

(150, 64, 64)

In [ ]:
import jraph
import jax.numpy as jnp

# Define a three node graph, each node has an integer as its feature.
node_features = jnp.array([[0.], [1.], [2.]])

# We will construct a graph for which there is a directed edge between each node
# and its successor. We define this with `senders` (source nodes) and `receivers`
# (destination nodes).
senders = jnp.array([0, 1, 2])
receivers = jnp.array([1, 2, 0])

# You can optionally add edge attributes.
edges = jnp.array([[5.], [6.], [7.]])

# We then save the number of nodes and the number of edges.
# This information is used to make running GNNs over multiple graphs
# in a GraphsTuple possible.
n_node = jnp.array([3])
n_edge = jnp.array([3])

# Optionally you can add `global` information, such as a graph label.

global_context = jnp.array([[1]])
graph = jraph.GraphsTuple(nodes=node_features, senders=senders, receivers=receivers,
edges=edges, n_node=n_node, n_edge=n_edge, globals=global_context)